In [1]:
import os

In [2]:
%pwd

'e:\\projects\\DL\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\projects\\DL\\Chicken-Disease-Classification'

In [34]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)  # Not mentioned then TypeError: PrepareCallbacksConfig() takes no arguments
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [35]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import create_directories, read_yaml


class ConfigutionManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        #create_directories([config.root_dir])
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config

In [36]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only= True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [38]:
# pipeline
try:
    config = ConfigutionManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config = prepare_callbacks_config)
    callbacks_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2025-06-23 15:07:01,077:  INFO:  common:  yaml file: config\config.yaml loaded successfully]


[2025-06-23 15:07:01,135:  INFO:  common:  yaml file: params.yaml loaded successfully]
[2025-06-23 15:07:01,170:  INFO:  common:  created directory at: artifacts]
[2025-06-23 15:07:01,183:  INFO:  common:  created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-06-23 15:07:01,189:  INFO:  common:  created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [33]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f'tb_logs_at_{timestamp}'

'tb_logs_at_2025-06-23-14-50-25'